In [271]:
#importing dataset
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
import hashlib
import re
nltk.download('punkt')
df = pd.read_csv('iris.csv',sep=',')
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [272]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.preprocessing import StandardScaler

In [273]:
df.isnull().value_counts()

sepal_length  sepal_width  petal_length  petal_width  species
False         False        False         False        False      150
dtype: int64

In [275]:
def feature_details(col, action):
    for l in data['design_state_data'][i][j].keys():
        #print(l)#//print
        if l=="feature_details":
            for m in data['design_state_data'][i][j][l].keys():
                #print(data['design_state_data'][i][j][l][m])
                if data['design_state_data'][i][j][l][m]=="Impute":
                    if (df.columns[0]==col) or (df.columns[2]==col):
                        df[j] = df[j].fillna(action)
                        #print('finished')
                    elif (df.columns[1]==col) or (df.columns[3]==col):
                        df[j]=df[j].fillna(data['design_state_data'][i][j][l]["impute_value"])
                        #print('finished')
                elif df.columns[4]==col:
                    for s in df['species']:
                        #print(s)
                        df['species']=hash(s)
                        break    
    
                #my_dict.values()).index(100)]

In [276]:
def algorithm_details(selected_algo):
    #print("algo details", selected_algo)
    reg=""
   # for k in data['design_state_data'][i][j].keys():
    for k in selected_algo.keys():
        temp = iter(selected_algo)
        if selected_algo[k]=="Random Forest Regressor":  
            #print("model_name",selected_algo[k])
            reg=RandomForestRegressor(max_depth=selected_algo['max_depth'], 
                                      min_samples_leaf=selected_algo['min_samples_per_leaf_min_value'])
            #print("Reg", reg)
        elif selected_algo[k]=="Gradient Boosted Trees":
            reg=GradientBoostingRegressor(subsample=selected_algo['min_subsample'], max_depth=selected_algo['max_depth'], 
                                          n_iter_no_change=selected_algo['min_iter'])
        elif selected_algo[k]=="LinearRegression":
            reg=LinearRegression(fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
        elif selected_algo[k]=="LogisticRegression":
            reg=LogisticRegression(max_iter=selected_algo['max_iter'], C=100, multi_class='ovr')
        elif selected_algo[k]=="RidgeRegression":
            reg=Ridge(max_iter=selected_algo['max_iter'])
        elif selected_algo[k]=="Lasso Regression":
            reg=Lasso(max_iter=selected_algo['max_iter'])
        elif selected_algo[k]=="Decision Tree":
            reg=DecisionTreeRegressor(max_depth=selected_algo['max_depth'])
    #print('Reg: ',reg)
    return reg


def train(algorithm):
    #print(df)
    X=df.drop(target_col_name, axis=1)
    #print(X)
    match = re.search(r'Logistic', str(algorithm))
    if match:
        y=df[target_col_name].astype('int')
        X=df.astype('int')
        sc=StandardScaler()
        sc.fit(X)
        X=sc.transform(X)
       # y=sc.transform(y).reshape(1, -1)
        #print(X)
        #print(y)
    else:
        y=df[target_col_name]
    #print("reg: ", algorithm)
    return X, y


def run_predict(algorithm):
    X, y=train(algorithm)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    algorithm.fit(X_train, y_train)
    y_pred=algorithm.predict(X_test)
    print("i) Predicted values ")
    print(y_pred)
    
    
    print("i) Before Hypertuning")
    metrics(y_test, y_pred)
    

def metrics(y_test, y_pred):
    mae = mean_absolute_error(y_true=y_test,y_pred=y_pred)
    #squared True returns MSE value, False returns RMSE value.
    mse = mean_squared_error(y_true=y_test,y_pred=y_pred) #default=True
    rmse = mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False)
    print("MAE:",mae)
    print("MSE:",mse)
    print("RMSE:",rmse)

def hyperparametering_GVgridSearch(algorithm, pos):
    # Define the grid 
    param={
    }
    param_grid = pos
    for a in param_grid: 
        #print("key: ", a, " val: ", [param_grid[a]])
        if a in list(algorithm.get_params().keys()):
            param[a]=[param_grid[a]]
            #param[a]=[param_grid[a]]
    
    #print("param_grid:", param)
        # Instantiate GridSearchCV
    model_gridsearch = GridSearchCV(
    estimator=algorithm, param_grid=param, scoring='neg_mean_squared_error', n_jobs=4, cv=5,
    refit=True, return_train_score=True)
    X, y=train(algorithm)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    #print(algorithm.get_params().keys())
    # Fit the selected model
    model_gridsearch.fit(X_train, y_train)
    # Print the time spend and number of models ran
    #print("GridSearchCV took %.2f seconds for %d candidate parameter settings." % ((time() - start), len(model_gridsearch.cv_results_['params'])))
    y_pred_grid = model_gridsearch.predict(X_test)
    print("ii) Predicted values using GridSearchCV")
    print(y_pred_grid)
    
    
    print("After Hypertuning using GridSearchCV")
    metrics(y_test, y_pred_grid)


In [277]:
import json
run_algo=""
target_col_name=""
selected="No Reduction"

with open('algoparams_from_ui.json') as json_file:
    data = json.load(json_file)
    #print(data.keys())
    if 'design_state_data' in data:
        #print(data['design_state_data'].keys())
        for i in data['design_state_data'].keys():
            if i=="target":
                print("1) Reading the Target variable and Type of regression to be done!!! ")
                for j in data['design_state_data'][i].keys():
                    if j=="target":
                        target_col_name=data['design_state_data'][i][j]
                        y=df.loc[:, target_col_name]
                        print('i)')
                        print("Target variable: ",target_col_name)
                        print("value: ", y)
                        #y=df[value]
                        #print(y)
                    elif j=="type":
                        value=data['design_state_data'][i][j]
                        df_type=value
                        print('iI)')
                        print(target_col_name+" of type : ",value)

            elif i=="feature_handling":
                print("2) Read the features (which are column names in the csv) and figure out what missing imputation needs to be applied and apply that to the columns loaded in a dataframe")
                print("i) ")
                print("Before handling feature :")
                print(df)
                for j in data['design_state_data'][i].keys():
                    #print(j)
                    if ((j==df.columns[0]) or (j==df.columns[2])):
                        action=df[j].mean()
                        feature_details(j, action)
                    elif ((j==df.columns[1]) or (j==df.columns[3])):
                        feature_details(j, 2)                        
                    elif j==df.columns[4]:
                        feature_details(j, 1)
                print("ii) ")
                print("After handling feature :")
                print(df)

            elif i=="feature_reduction":
                print("3) Compute feature reduction based on input. See the screenshot below where there can be No Reduction, Corr with Target, Tree-based, PCA.")
                print("i) ")
                print("Before reducing feature :")
                print(df)
                for j in data['design_state_data'][i].keys():
                    if not type(data['design_state_data'][i][j])==str:
                        #print("j: ",j,"  selected:",selected)
                        if j==selected:
                            for k in data['design_state_data'][i][j].keys():
                                if data['design_state_data'][i][j][k] == True:
                                    df=df.iloc[:, 0:5]
                                elif data['design_state_data'][i][j][k] == False:
                                    df=df.iloc[:,0:0]
                print("ii) After reducing feature :")
                print(df)
                            #print(data['design_state_data'][i][j].keys())
                            #if data['design_state_data'][i][j].keys()==str:
                           
            elif i=='algorithms': 
                print("4) Parse the Json and make the model objects (using sklean) that can handle what is required in the “prediction_type” specified in the JSON (See #1 where “prediction_type” is specified). Keep in mind not to pick models that don’t apply for the prediction_type specified")
                for j in data['design_state_data'][i].keys():
                    df_type=str(df_type)
                    match = re.search(r'egress', j)
                    #print(match)
                    if match:
                        #print("Found!")
                        #model building
                        temp=data['design_state_data'][i][j]
                        print('Selected regression Algorithm: ',j)
                        run_algo=algorithm_details(temp)
                        print("5) Run the fit and predict on each model – keep in mind that you need to do hyper parameter tuning i.e., use GridSearchCV")
                        run_predict(run_algo)
                        position=data['design_state_data']['hyperparameters']
                        hyperparametering_GVgridSearch(run_algo, position)
                    else:
                        print(j,' is Not a Regression Algorithm')
               
            '''
            elif i=='hyperparameters':
                for j in data['design_state_data'][i].keys():
                    hyperparametering_GVgridSearch(run_algo)
            
            else:
                break
                '''

1) Reading the Target variable and Type of regression to be done!!! 
i)
Target variable:  petal_width
value:  0      0.2
1      0.2
2      0.2
3      0.2
4      0.2
      ... 
145    2.3
146    1.9
147    2.0
148    2.3
149    1.8
Name: petal_width, Length: 150, dtype: float64
iI)
petal_width of type :  regression
2) Read the features (which are column names in the csv) and figure out what missing imputation needs to be applied and apply that to the columns loaded in a dataframe
i) 
Before handling feature :
     sepal_length  sepal_width  petal_length  petal_width         species
0             5.1          3.5           1.4          0.2     Iris-setosa
1             4.9          3.0           1.4          0.2     Iris-setosa
2             4.7          3.2           1.3          0.2     Iris-setosa
3             4.6          3.1           1.5          0.2     Iris-setosa
4             5.0          3.6           1.4          0.2     Iris-setosa
..            ...          ...           .

i) Predicted values 
[2.21781565 0.24458867 0.11910839 1.99316705 2.0019925  0.34876817
 0.3136427  0.23880286 1.70602487 1.49111711 0.19333797 0.26390597
 2.07575257 2.2917401  1.84778282 1.57948844 1.16580983 1.98375496
 1.6115626  0.32291403 0.24458867 1.61355776 1.71590647 1.21933721
 1.77477377 0.25416514 0.22618676 1.93933454 1.60337103 1.9727936
 1.77324812 1.47318471 1.62435475 0.20503867 1.80214188 1.76489217
 1.49812345 2.08488315]
i) Before Hypertuning
MAE: 0.1677142041431399
MSE: 0.0421957358455294
RMSE: 0.2054160067899515
ii) Predicted values using GridSearchCV
[1.20094699 0.16991827 0.99734295 2.07063412 0.37946082 1.51761723
 1.39085206 1.09675848 1.8164355  0.15403313 0.07684191 1.77949159
 0.99110038 1.23900814 0.19691555 2.03557232 1.34431388 2.15229427
 0.03020713 1.76213686 1.91214697 1.60034631 1.98081286 0.25568312
 1.61359634 1.12888111 0.21386963 0.32546624 0.25456589 0.8281926
 2.33290913 1.99527671 0.40163682 1.47214799 1.2305311  1.57116286
 1.81126186 0.2408